# 07. Data Subsetting and Transformation Services in the Cloud

## Using the Harmony-Py library to access customized data from NASA Earthdata 

---

### Timing:
- Exercise: 40 minutes

---

## Summary

We have already explored direct access to the NASA EOSDIS archive in the cloud via the Amazon Simple Storage Service (S3) by using the Common Metadata Repository (CMR) to search for granule locations. In addition to directly accessing the files archived and distributed by each of the NASA DAACs, many datasets also support services that allow us to customize the data via subsetting, reformatting, reprojection, and other transformations. 

This tutorial demonstrates how to find, request, and use customized data from a new ecosystem of services operating within the NASA Earthdata Cloud: NASA Harmony.

### Benefits

But first, why use this option when we've already learned how to access data directly from the NASA Earthdata Cloud? 

- Consistent access patterns to EOSDIS holdings make cross-data center data access easier
- Data reduction services allow us to request only the data we want, in the format and projection we want
- Analysis Ready Data and cloud access will help reduce time-to-science
- Community Development helps reduce the barriers for re-use of code and sharing of domain knowledge

![Data file filtering and subsetting](data/subsetting_diagram.png)

See more on the [Earthdata Harmony landing page](https://harmony.earthdata.nasa.gov/), including documentation on the Harmony API itself. 


### Objectives

1. Conceptualize the data transformation service types and offerings provided by NASA Earthdata, including Harmony
2. Practice skills learned from the introductory CMR tutorial to discover what access and service options exist for a given data set, as well as variable metadata.
2. 
___

## Import Packages

In [1]:
from harmony import BBox, Client, Collection, Request, LinkType
from harmony.config import Environment
import requests
from pprint import pprint
import datetime as dt
import s3fs
import xarray as xr

## Discover service options for a given data set

#### _First, what do we mean by a "service"?_

In the context of NASA Earthdata, we are usually referring to a service as any data transformation or customization process that packages or delivers data in a way that makes it easier to work with compared to how the data are natively archived at NASA EOSDIS. Basic customization options may include:
* Subsetting (cropping) the data by:
    * Variable
    * Spatial boundary,
    * Temporal range
* Reformatting
    * For example: From NetCDF-4 to Cloud Optimized GeoTIFF
* Reprojection and/or Resampling
    * For example: From Sinusoidal to Polar Stereographic
* Mosaicking
* Aggregating

A few main types or pathways for services that are commonly supported across the NASA DAACs include:
* [NASA Global Imagery Browse Service](https://earthdata.nasa.gov/eosdis/science-system-description/eosdis-components/gibs)
    * Web services providing imagery, much of which is updated daily, to broaden accessibility of NASA EOSDIS data to the media and public.
        * Web Map Tile Service (WMTS)
        * Tiled Web Map Service (TWMS)
        * Web Map Service (WMS)
        * Keyhole Markup Language (KML)
        * Geospatial Data Abstraction Library (GDAL)
* [OPeNDAP](https://opendap.earthdata.nasa.gov/)
    * The Open-source Project for a Network Data Access Protocol is a NASA community standard DAP that provides a simple way to access and work with data over the internet. OPeNDAP's client/server software allows us to subset and reformat data using an internet browser, command line interface, and other applications.
* [Harmony](https://harmony.earthdata.nasa.gov/)
    * In the most basic sense, Harmony is an Application Programming Interface, or API, allowing us to request customization options described above, which are then processed and returned as file outputs. Harmony helps to reduce pre-processing steps so we can spend less time preparing the data, and more time doing science.  

**Note: These service offerings are unique to each NASA EOSDIS dataset.**

Why is this?

Due to varying [levels of service](https://earthdata.nasa.gov/collaborate/new-missions/level-of-service), cloud migration status, and unique characteristics of the datasets themselves, not all service options are provided for all datasets. Therefore it is important to first explore a given dataset's metadata to discover what service options are provided.

Let's utilize the CMR API skills we learned on Day 1 to inspect service metadata:

In [4]:
url = 'https://cmr.earthdata.nasa.gov/search'

We want to search by collection to inspect the access and service options that exist:

In [5]:
collection_url = f'{url}/{"collections"}'

In the CMR introduction tutorial, we explored cloud-hosted collections from different DAAC providers, and identified the CMR `concept-id` for a given data set id (also referred to as a `short_name`). 

Here we are jumping ahead and already know the `concept_id` we are interested in, by browsing cloud-hosted datasets from PO.DAAC in Earthdata Search: https://search.earthdata.nasa.gov/portal/podaac-cloud/search. 

We are going to focus on `MODIS_A-JPL-L2P-v2019.0`: GHRSST Level 2P Global Sea Surface Skin Temperature from the Moderate Resolution Imaging Spectroradiometer (MODIS) on the NASA Aqua satellite (GDS2). Let's first save this as a variable that we can use later on once we request data from Harmony.

In [6]:
short_name= 'MODIS_A-JPL-L2P-v2019.0'
concept_id = 'C1940473819-POCLOUD'

We will view the top-level metadata for this collection to see what additional service and variable metadata exist.

In [21]:
response = requests.get(collection_url, 
                        params={
                            'concept_id': concept_id,
                            },
                        headers={
                            'Accept': 'application/json'
                            }
                       )
response = response.json()

Print the response:

In [8]:
pprint(response)

{'feed': {'entry': [{'archive_center': 'NASA/JPL/PODAAC',
                     'associations': {'services': ['S1962070864-POCLOUD',
                                                   'S2004184019-POCLOUD'],
                                      'tools': ['TL2108419875-POCLOUD',
                                                'TL2092786348-POCLOUD'],
                                      'variables': ['V1997812737-POCLOUD',
                                                    'V1997812697-POCLOUD',
                                                    'V2112014688-POCLOUD',
                                                    'V1997812756-POCLOUD',
                                                    'V1997812688-POCLOUD',
                                                    'V1997812670-POCLOUD',
                                                    'V1997812724-POCLOUD',
                                                    'V2112014684-POCLOUD',
                                                

#### What do each of these service values mean?

- Associations
     - CMR is a large web of interconnected metadata "schemas", including Collections, Granules, Services, Tools, and Variables. In this case, this collection is associated with two unique services, two tools, and several unique variables.
- Tags
    - There are also tags that describe what service options exist at a high-level. In this case, we see that this dataset supports the ability to reformat, subset by space and time, as well as by variable. This is used in web applications like Earthdata Search to surface those customization options more readily. 
- Service Features
    - In this case, we see three separate "features" listed here: esi, Harmony, and OPeNDAP.
    
We will dig into more details on what Harmony offers for this dataset.

First, we need to isolate the services returned for this dataset:

In [9]:
services = response['feed']['entry'][0]['associations']['services']
print(services)

['S1962070864-POCLOUD', 'S2004184019-POCLOUD']


In [10]:
service_url = "https://cmr.earthdata.nasa.gov/search/services"

Inspect the first service returned. Now we're going to search the services endpoint to view that individual service's metadata, like we did with our dataset above. This time, we're explicitly setting the format of the response to umm-json in the Accept Header in order to see detailed metadata about the service.

In [22]:
service_response = requests.get(service_url, 
                        params={
                            'concept_id': services[0],
                            },
                        headers={
                            'Accept': 'application/vnd.nasa.cmr.umm_results+json'
                            }
                       )
service_response = service_response.json()

In [23]:
pprint(service_response)

{'hits': 1,
 'items': [{'meta': {'concept-id': 'S1962070864-POCLOUD',
                     'concept-type': 'service',
                     'deleted': False,
                     'format': 'application/vnd.nasa.cmr.umm+json',
                     'native-id': 'POCLOUD_podaac_l2_cloud_subsetter',
                     'provider-id': 'POCLOUD',
                     'revision-date': '2021-11-02T22:57:03.597Z',
                     'revision-id': 19,
                     'user-id': 'podaaccloud'},
            'umm': {'AccessConstraints': 'None',
                    'Description': 'Endpoint for subsetting L2 Subsetter via '
                                   'Harmony',
                    'LongName': 'PODAAC Level 2 Cloud Subsetter',
                    'MetadataSpecification': {'Name': 'UMM-S',
                                              'URL': 'https://cdn.earthdata.nasa.gov/umm/service/v1.4',
                                              'Version': '1.4'},
                    'Name': 'PO

TODO: Describe these different service options and broader Harmony / backend subsetter context.

## Discover variable names

TODO: Could this be an "exercise" to gain more familiarity with CMR? 

In [24]:
variables = response['feed']['entry'][0]['associations']['variables']
print(variables)

['V1997812737-POCLOUD', 'V1997812697-POCLOUD', 'V2112014688-POCLOUD', 'V1997812756-POCLOUD', 'V1997812688-POCLOUD', 'V1997812670-POCLOUD', 'V1997812724-POCLOUD', 'V2112014684-POCLOUD', 'V1997812701-POCLOUD', 'V1997812681-POCLOUD', 'V2112014686-POCLOUD', 'V1997812663-POCLOUD', 'V1997812676-POCLOUD', 'V1997812744-POCLOUD', 'V1997812714-POCLOUD']


In [25]:
var_url = "https://cmr.earthdata.nasa.gov/search/variables"

In [26]:
var_response = requests.get(f"{var_url}.{output_format}?concept-id={variables[0]}")

In [27]:
pprint(var_response.json())

{'hits': 1,
 'items': [{'associations': {'collections': [{'concept-id': 'C1940473819-POCLOUD'}]},
            'meta': {'concept-id': 'V1997812737-POCLOUD',
                     'concept-type': 'variable',
                     'deleted': False,
                     'format': 'application/vnd.nasa.cmr.umm+json',
                     'native-id': 'MODIS_A-JPL-L2P-v2019.0-sses_standard_deviation_4um',
                     'provider-id': 'POCLOUD',
                     'revision-date': '2021-10-19T02:26:51.560Z',
                     'revision-id': 6,
                     'user-id': 'jmcnelis'},
            'umm': {'DataType': 'byte',
                    'Definition': 'mid-IR SST standard deviation error; non '
                                  'L2P core field; signed byte array:  WARNING '
                                  'Some applications are unable to properly '
                                  'handle signed byte values. If values are '
                                  'encountered 

Let's print out a simple list of all associated variable names.

In [29]:
var_list = []
for i in range(len(variables)):
    response = requests.get(f"{var_url}.{output_format}?concept-id={variables[i]}")
    response = response.json()
    var_list.append(response['items'][0]['umm']['Name'])

In [30]:
pprint(var_list)

['sses_standard_deviation_4um',
 'l2p_flags',
 'time',
 'dt_analysis',
 'sses_standard_deviation',
 'sst_dtime',
 'sses_bias_4um',
 'lat',
 'sea_surface_temperature_4um',
 'sses_bias',
 'lon',
 'sea_surface_temperature',
 'quality_level',
 'wind_speed',
 'quality_level_4um']


## Find all collections that have harmony type??

## Harmony-Py set up

[TODO] Describe Harmony-Py in more detail (connection between the library and the API).

Harmony-Py is a Python library for integrating with NASA's Harmony Services.

Harmony-Py provides a Python alternative to directly using Harmony's RESTful API. It handles NASA Earthdata Login (EDL) authentication and optionally integrates with the CMR Python Wrapper by accepting collection results as a request parameter. It's convenient for scientists who wish to use Harmony from Jupyter notebooks as well as machine-to-machine communication with larger Python applications.

Next steps adopted from the intro tutorial notebook in the Harmony-Py library: https://github.com/nasa/harmony-py/blob/main/examples/intro_tutorial.ipynb

### Create Harmony Client object
First, we need to create a Harmony Client, which is what we will interact with to submit and inspect a data request to Harmony, as well as to retrieve results.

When creating the Client, we need to provide Earthdata Login credentials, which are required to access data from NASA EOSDIS. This basic line below assumes that we have a `.netrc` available. 

In [ ]:
harmony_client = Client()

### Create Harmony Request
The following are common request parameters:

- `collection`: Required parameter. This is the NASA EOSDIS collection, or data product. There are two options for inputting a collection of interest:
    - Provide a concept ID, which is an ID provided in the Common Metadata Repository (CMR) metadata
    - Data product short name (e.g. SENTINEL-1_INTERFEROGRAMS).
- `spatial`: Bounding box spatial constraints on the data. The Harmony Bbox class accepts spatial coordinates as decimal degrees in w, s, e, n order, where longitude = -180, 180 and latitude = -90, 90.
- `temporal`: Date/time constraints on the data. The example below demonstrates temporal start and end ranges using the python datetime library.

Other advanced parameters that may be of interest. Note that many reformatting or advanced projection options may not be available for your requested dataset. See the [documentation](https://harmony-py.readthedocs.io/en/latest/) for details on how to construct these parameters.

[TODO] Describe area/time use case with this dataset - maybe demonstrating Earthdata Search for browsing??

We will search over the Gulf of Mexico during the period 1 Aug 2020 to 1 Nov 2020, since we are interested in exploring how SST responds during the latest Atlantic hurricane season.

In [ ]:
request = Request(
    collection=Collection(id=short_name),
#    spatial=BBox(60,-45.75,90,-45), # bounding box example that can be used as an alternative to shapefile input
    spatial=BBox(-97.77667,21.20806,-83.05197,30.16605), # bounding box example that can be used as an alternative to shapefile input
    temporal={
        # 'start': dt.datetime(2021, 11, 1),
        # 'stop': dt.datetime(2021, 11, 2),
        'start': dt.datetime(2021, 8, 1),
        'stop': dt.datetime(2021, 8, 2),
    },
    # variables=variables,
)

### Check Request validity
Before submitting a Harmony Request, we can test your request to see if it's valid and how to fix it if not. In particular, `request.is_valid` will check to ensure that the spatial BBox bounds and temporal ranges are entered correctly.

In [ ]:
request.is_valid()

### Submit request

Now that the request is created, we can now submit it to Harmony using the Harmony Client object. A job id is returned, which is a unique identifier that represents the submitted request.

In [ ]:
job_id = harmony_client.submit(request)
job_id

### Check request status

We can check on the progress of a processing job with status(). This method blocks while communicating with the server but returns quickly.

In [ ]:
harmony_client.status(job_id)

Depending on the size of the request, it may be helpful to wait until the request has completed processing before the remainder of the code is executed. The wait_for_processing() method will block subsequent lines of code while optionally showing a progress bar.

In [ ]:
harmony_client.wait_for_processing(job_id, show_progress=True)

### View Harmony job response and output URLs
Once the data request has finished processing, we can view details on the job that was submitted to Harmony, including the API call to Harmony, and informational messages on the request if available.

result_json() calls wait_for_processing() and returns the complete job in JSON format once processing is complete. 

In [ ]:
data = harmony_client.result_json(job_id)
pprint(data)

### Direct cloud access

Note that the remainder of this tutorial will only succeed when running this notebook within the AWS us-west-2 region.

Harmony data outputs can be accessed within the cloud using the s3 URLs and AWS credentials provided in the Harmony job response.

#### Retrieve list of output URLs.

The result_urls() method calls wait_for_processing() and returns a list of the processed data URLs once processing is complete. You may optionally show the progress bar as shown below.

In [ ]:
results = harmony_client.result_urls(job_id, link_type=LinkType.s3)
urls = list(results)
pprint(urls)

We can see that the first file returned does not include the `_subsetted` suffix, which indicates that a blank file was returned, as no data values were located within our subsetted region. We'll select the second URL in the list to bring into xarray below.

In [ ]:
url = urls[1]
url

#### AWS credential retrieval
Using aws_credentials you can retrieve the credentials needed to access the Harmony s3 staging bucket and its contents.

In [ ]:
creds = harmony_client.aws_credentials()

### Open staged files with *s3fs* and *xarray*

We use the AWS `s3fs` package to create a file system that can then be read by xarray:

In [ ]:
s3_fs = s3fs.S3FileSystem(
    key=creds['aws_access_key_id'],
    secret=creds['aws_secret_access_key'],
    token=creds['aws_session_token'],
    client_kwargs={'region_name':'us-west-2'},
)

### **Note, Ideally, we'd show how to bring in all files into a single xarray ds but I'm getting an error:

In [ ]:
# # Iterate through remote_files to create a fileset
# fileset = [s3_fs.open(file) for file in urls]
# fileset
# ds = xr.open_mfdataset(fileset)

Now that we have our s3 file system set, including our declared credentials, we'll use that to open the url, and read in the file through xarray:

In [ ]:
f = s3_fs.open(url, mode='rb')
# ds = xr.open_dataset(f, decode_cf=False)
ds = xr.open_dataset(f)
ds.sea_surface_temperature.plot()
f.close()

### Plot the data

In [ ]:
# # Determine the lat/lon coordinate names
# for coord_name, coord in ds.coords.items():
#     if 'units' not in coord.attrs:
#         continue
#     if coord.attrs['units'] == 'degrees_north':
#         lat_var = coord_name
#     if coord.attrs['units'] == 'degrees_east':
#         lon_var = coord_name

# print(f'lat_var={lat_var}')
# print(f'lon_var={lon_var}')

# # if ds[variable].size == 0:
# #     print("No data in subsetted region. Exiting")
# #     sys.exit(0)
    
# import matplotlib.pyplot as plt
# import math

# fig, axes = plt.subplots(ncols=3, nrows=math.ceil((len(ds.data_vars)/3)))
# fig.set_size_inches((15,15))

# for count, xvar in enumerate(ds.data_vars):
#     if  ds[xvar].dtype == "timedelta64[ns]":
#         continue
#         #ds[xvar].astype('timedelta64[D]').plot(ax=axes[int(count/3)][count%3])
#     ds[xvar].plot(ax=axes[int(count/3)][count%3])

### Extra code attempting to read in directly from s3 into xarray:

In [ ]:
# netcdf_stores = [netcdf_fs.get_mapper(root=u, check=False) for u in urls]
# netcdf_stores

# xr.open_mfdataset(netcdf_stores)

## Resources

https://github.com/podaac/tutorials/blob/master/notebooks/SWOT-EA-2021/Colocate_satellite_insitu_ocean.ipynb

